# GSY-e SDK for humans...:D

The purpose of this notebook is to manually create an asset and connect it to a \
running GSY instance running with a redis connection. "how difficult can it be".


#This is the SDK, assuming the installation went as expected, following the github advice
#there should be all the classses readily available and importable

Specifically, we did this before coming here:
    
git submodule update --init --recursive
pip install -e . -e gsy-framework

Do we have the libraries now?



The documentation states:
The constructor of the API class can connect and register automatically to a running collaboration.

And then it creates

asset_client = RedisAssetClient(<asset-uuid>, autoregister=True)
    
can we even import this RedisAssetClient?
    
By digging this is what I could do...

In [4]:
from simple_oracles.bc4p_simple_oracle import Oracle
from typing import List, Dict
from time import sleep
from gsy_e_sdk.redis_aggregator import RedisAggregator
from gsy_e_sdk.clients.redis_asset_client import RedisAssetClient

#I an Oracle to be able to model things at will
#An oracle is a RedisAggregator. A ReddisAggregator is an interface to deal
#with a set of assets by name (see below), it leaves some methods not implemented
#The oracle in  simple_oracles.bc4p_simple_oracle impelemts them
#I am reorganizing the code so that it is understandable by a human...

ORACLE_NAME = "oracle" #just a the name of the oracle, to call it something

# List of assets' names to be connected with the API
LOAD_NAMES = ["Load 1 L13", "Load 2 L21", "Load 3 L17"] #assets name, we need these to register to gsy assets
PV_NAMES = ["PV 1 (4kW)", "PV 3 (5kW)"]
STORAGE_NAMES = ["Tesla Powerwall 3"]

#this oracle concept is also what we have to use to access external data, such as influxdb


# Frequency of bids/offers posting in a market slot - to leave as it is
TICK_DISPATCH_FREQUENCY_PERCENT = 10

aggregator = Oracle(aggregator_name=ORACLE_NAME,TICK_DISPATCH_FREQUENCY_PERCENT=TICK_DISPATCH_FREQUENCY_PERCENT)
#aggregator.TICK_DISPATCH_FREQUENCY_PERCENT = 10

asset_args = {"autoregister": True, "pubsub_thread": aggregator.pubsub}


def register_asset_list(asset_names: List, asset_params: Dict, asset_uuid_map: Dict) -> Dict:
    """Register the provided list of assets with the aggregator."""
    for asset_name in asset_names:
        print("Registered asset:", asset_name)
        asset_params["area_id"] = asset_name
        asset = RedisAssetClient(**asset_params)# what if there is no market running?
        asset_uuid_map[asset.area_uuid] = asset.area_id
        asset.select_aggregator(aggregator.aggregator_uuid)
    return asset_uuid_map


print()
print("Registering assets ...")
asset_uuid_mapping = {}
asset_uuid_mapping = register_asset_list(LOAD_NAMES + PV_NAMES + STORAGE_NAMES,
                                         asset_args, asset_uuid_mapping)
print()
print("Summary of assets registered:")
print()
print(asset_uuid_mapping)

# loop to allow persistence
while not aggregator.is_finished:
    sleep(0.5)



Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/tmaltus/anaconda3/envs/bc4p/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/tmaltus/anaconda3/envs/bc4p/lib/python3.10/site-packages/redis/client.py", line 1757, in run
    pubsub.get_message(ignore_subscribe_messages=True, timeout=sleep_time)
  File "/home/tmaltus/anaconda3/envs/bc4p/lib/python3.10/site-packages/redis/client.py", line 1650, in get_message
    return self.handle_message(response, ignore_subscribe_messages)
  File "/home/tmaltus/anaconda3/envs/bc4p/lib/python3.10/site-packages/redis/client.py", line 1715, in handle_message
    handler(message)
  File "/media/tmaltus/12bfaee0-ff49-422e-a39b-d466076d3d11/BC4P/gsy-e-sdk/gsy_e_sdk/redis_client_base.py", line 113, in _on_register
    self._check_buffer_message_matching_command_and_id(message)
  File "/media/tmaltus/12bfaee0-ff49-422e-a39b-d466076d3d11/BC4P/gsy-e-sdk/gsy_e_sdk/redis_client_base.py", line 65


Registering assets ...
Registered asset: Load 1 L13
Registered asset: Load 2 L21
Registered asset: Load 3 L17
Registered asset: PV 1 (4kW)
Registered asset: PV 3 (5kW)
Registered asset: Tesla Powerwall 3

Summary of assets registered:

{'2d628281-c56e-414d-b83c-afdd21ca8c88': 'Load 1 L13', 'a4e1885c-8416-4276-a1ed-94b61cf452ed': 'Load 2 L21', '60997325-2dc2-44ac-bf66-c97d67627739': 'Load 3 L17', '3760366e-3306-48e5-b31d-6fffb5cde3dc': 'PV 1 (4kW)', 'ec814715-0c8a-4e2e-b1ab-7b6dec51afec': 'PV 3 (5kW)', '5b14e9e1-75ba-4c44-9730-8672bcc436a9': 'Tesla Powerwall 3'}


KeyboardInterrupt: 

basic API stuff taken from "redis_basic stragies, an asset has to be created for each of the registered assets
the mapping is done through the id of the asset.
everything then happens in oracles.